## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Johnson City,US,36.3134,-82.3535,75.43,73,20,0.00,few clouds
1,1,Bria,CF,6.5423,21.9863,66.27,98,100,1.25,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,55.74,62,27,11.68,scattered clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,65.17,61,0,4.00,clear sky
4,4,Tay Ninh,VN,11.3000,106.1000,87.22,66,5,6.49,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Johnson City,US,36.3134,-82.3535,75.43,73,20,0.00,few clouds
4,4,Tay Ninh,VN,11.3000,106.1000,87.22,66,5,6.49,clear sky
6,6,Russell,US,32.3502,-85.1999,82.90,74,0,0.00,clear sky
11,11,Sao Filipe,CV,14.8961,-24.4956,78.48,77,2,15.03,clear sky
12,12,Terenos,BR,-20.4422,-54.8603,74.91,46,0,9.22,clear sky
14,14,Rikitea,PF,-23.1203,-134.9692,73.29,74,9,20.67,light rain
15,15,Vaini,TO,-21.2000,-175.2000,80.76,89,20,8.05,few clouds
18,18,Heihe,CN,50.2441,127.4902,73.98,96,100,4.52,moderate rain
25,25,Samarai,PG,-10.6167,150.6667,79.25,85,68,13.89,broken clouds
26,26,Shingu,JP,33.7333,135.9833,85.62,70,41,8.81,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                346
City                   346
Country                346
Lat                    346
Lng                    346
Max Temp               346
Humidity               346
Cloudiness             346
Wind Speed             346
Weather Description    346
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Johnson City,US,36.3134,-82.3535,75.43,73,20,0.00,few clouds
4,4,Tay Ninh,VN,11.3000,106.1000,87.22,66,5,6.49,clear sky
6,6,Russell,US,32.3502,-85.1999,82.90,74,0,0.00,clear sky
11,11,Sao Filipe,CV,14.8961,-24.4956,78.48,77,2,15.03,clear sky
12,12,Terenos,BR,-20.4422,-54.8603,74.91,46,0,9.22,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Johnson City,US,75.43,few clouds,36.3134,-82.3535,
4,Tay Ninh,VN,87.22,clear sky,11.3000,106.1000,
6,Russell,US,82.90,clear sky,32.3502,-85.1999,
11,Sao Filipe,CV,78.48,clear sky,14.8961,-24.4956,
12,Terenos,BR,74.91,clear sky,-20.4422,-54.8603,
14,Rikitea,PF,73.29,light rain,-23.1203,-134.9692,
15,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
18,Heihe,CN,73.98,moderate rain,50.2441,127.4902,
25,Samarai,PG,79.25,broken clouds,-10.6167,150.6667,
26,Shingu,JP,85.62,scattered clouds,33.7333,135.9833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add parameters to search for hotels with 5000 meters.
    params = {
    "location": f"{lat}, {lng}",
    "radius": 5000,
    "type": "lodging",
    "key": g_key }

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
print(fig)

Figure(layout=FigureLayout(height='420px'))
